<img src="tmva_logo.gif" height="20%" width="20%">


# TMVA  Reader Example 

#### Example of applying the trained classified of TMVA on a data set and evaluate the classifier

In [ ]:
TMVA::Tools::Instance();

### Choose TMVA method to evaluate

In [ ]:
std::vector<std::string> methodNames = {"BDT","DL_CPU"};

###  Give input file 


In [ ]:
TFile * inputFile = TFile::Open("Higgs_data.root");

In [ ]:
inputFile->ls();


In [ ]:
TTree* sig_tree = (TTree*) inputFile->Get("sig_tree");
TTree* bkg_tree = (TTree*) inputFile->Get("bkg_tree");

In [ ]:
//sig_tree->Print(); 

### Declare Reader


In [ ]:
TMVA::Reader reader( "!Color:!Silent" );

### Add Variable

We add variables to the reader exactly in the same way we did for the **DataLoader** during the training

We need to specify the address of the variable in order to pass it to TMVA when we iterate on the TTree


In [ ]:
float m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb; 

In [ ]:
// add variables 
reader.AddVariable("m_jj",&m_jj);
reader.AddVariable("m_jjj",&m_jjj);
reader.AddVariable("m_lv",&m_lv);
reader.AddVariable("m_jlv",&m_jlv);
reader.AddVariable("m_bb",&m_bb);
reader.AddVariable("m_wbb",&m_wbb);
reader.AddVariable("m_wwbb",&m_wwbb);

## Setup Classifiers

We set up the classifiers by reading the input weights from the appropriate files

The file is stored for example as 

####  *dataset/weights/TMVAClassification_BDT.weights.xml

In [ ]:
for (size_t i = 0; i < methodNames.size(); ++i) { 
   TString weightfile = "dataset/weights/TMVA_Higgs_Classification_" + methodNames[i] + ".weights.xml";
   reader.BookMVA( methodNames[i], weightfile );
}

### Book histogram to visualize the response

In [ ]:
auto h1 = new TH1D("h1","Classifier Output on Background Events",100,-1,1);
auto h2 = new TH1D("h2","Classifier Output on Signal Events",100,-1,1);
std::vector<std::vector<float>> vout(methodNames.size());

## Iterate on the events from the input TTree 

In [ ]:
auto readEvents = [&](TTree & tree, TH1 & histo) {
  tree.SetBranchAddress("m_jj",&m_jj);
  tree.SetBranchAddress("m_jjj",&m_jjj);
  tree.SetBranchAddress("m_lv",&m_lv) ;
  tree.SetBranchAddress("m_jlv",&m_jlv);
  tree.SetBranchAddress("m_bb",&m_bb);
  tree.SetBranchAddress("m_wbb",&m_wbb);
  tree.SetBranchAddress("m_wwbb",&m_wwbb);
  for (size_t i = 0; i < methodNames.size(); ++i ) {
    vout[i].clear(); 
    vout[i].resize(tree.GetEntries());
  }
  for (Long64_t ievt=0; ievt < tree.GetEntries();ievt++) {

      if (ievt%500 == 0) std::cout << "--- ... Processing event: " << ievt << std::endl;

      tree.GetEntry(ievt);
      
      for (size_t i = 0; i < methodNames.size(); ++i ) {
    
         auto output = reader.EvaluateMVA(methodNames[i]);
  
         vout[i][ievt] = output;  
      }
  }
};
    

### Evaluation of the classifier on the given input events

We iterate on the input event in the given TTree. We provide as input first the background tree 
We need first to define the branches we are reading from the trees and then we can read the events calling 
the readEvents function we have defined before

In [ ]:
// read events and fill histogram
readEvents(*bkg_tree,*h1); 

In [ ]:
std::vector<std::vector<float>> bkg_results = vout; 

### Evaluation on the signal  events

We evaluate now the classifier on the another tree (e.g. the signal events)



In [ ]:
readEvents(*sig_tree,*h2); 

In [ ]:
std::vector<std::vector<float>> sig_results = vout; 

## Make ROC plot

In [ ]:
for (size_t i = 0; i < methodNames.size(); ++i) {
    TMVA::ROCCurve * roc = new TMVA::ROCCurve(sig_results[i], bkg_results[i] );
    TGraph *  roc_graph  = roc->GetROCCurve(); 
    roc_graph->SetTitle(methodNames[i].c_str());
    std::cout << "method " << methodNames[i] << " ROC integral = " << roc->GetROCIntegral() << std::endl;
    roc_graph->SetLineColor(i+1);
    if (i == 0) roc_graph->Draw("AL");
    else roc_graph->Draw("L");
}

In [ ]:
if (gPad) gPad->Draw();